<a href="https://colab.research.google.com/github/saithanmaisri/DocuQuery-AI-Powered-PDF-Knowledge-Assistant-Using-Google-PALM/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install pdfplumber google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 68.3 MB/s eta 0:00:00


In [ ]:
import pdfplumber
import google.generativeai as palm

In [ ]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text


In [ ]:
import pandas as pd

def analyze_price_list(text):
    # Dummy logic for demo
    # Ideally, use regex or pattern to extract item-price
    items = []
    for line in text.split("\n"):
        if "Rs" in line or "₹" in line:
            items.append(line.strip())
    return pd.DataFrame(items, columns=["Item Info"])


In [ ]:
def summarize_text_with_palm(text):
    response = palm.generate_text(
        model="models/text-bison-001",
        prompt=f"Summarize this research paper:\n\n{text[:3000]}",  # Limit input
        temperature=0.5
    )
    return response.result


In [ ]:
def ask_question(text, question):
    prompt = f"Based on this paper:\n{text[:3000]}\n\nAnswer this:\n{question}"
    response = palm.generate_text(
        model="models/text-bison-001",
        prompt=prompt,
        temperature=0.4
    )
    return response.result


In [ ]:
def match_resume_with_jd(resume_text, job_description):
    prompt = f"""
Job Description:
{job_description}

Resume:
{resume_text}

Evaluate how well this candidate matches the JD. Give a percentage match and key strengths.
"""
    response = palm.generate_text(
        model="models/text-bison-001",
        prompt=prompt,
        temperature=0.5
    )
    return response.result


In [ ]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.9 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import pdfplumber
import pandas as pd
import google.generativeai as palm

def extract_text_from_pdf(pdf_file):
    text = ""
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def analyze_price_list(text):
    items = []
    for line in text.split("\n"):
        if "Rs" in line or "₹" in line:
            items.append(line.strip())
    return pd.DataFrame(items, columns=["Item Info"])

def summarize_text_with_palm(text):
    response = palm.generate_text(
        model="models/text-bison-001",
        prompt=f"Summarize this research paper:\n\n{text[:3000]}",
        temperature=0.5
    )
    return response.result

def ask_question(text, question):
    prompt = f"Based on this paper:\n{text[:3000]}\n\nAnswer this:\n{question}"
    response = palm.generate_text(
        model="models/text-bison-001",
        prompt=prompt,
        temperature=0.4
    )
    return response.result

def match_resume_with_jd(resume_text, job_description):
    prompt = f"""
Job Description:
{job_description}

Resume:
{resume_text}

Evaluate how well this candidate matches the JD. Give a percentage match and key strengths.
"""
    response = palm.generate_text(
        model="models/text-bison-001",
        prompt=prompt,
        temperature=0.5
    )
    return response.result

st.title("DocuQuery: AI-Powered PDF Knowledge Assistant")

option = st.selectbox("Choose Scenario", ["Price List", "Research Paper", "Resume Matcher"])

uploaded_file = st.file_uploader("Upload PDF File", type=["pdf"])

if uploaded_file:
    pdf_text = extract_text_from_pdf(uploaded_file)

    if option == "Price List":
        df = analyze_price_list(pdf_text)
        st.dataframe(df)

    elif option == "Research Paper":
        summary = summarize_text_with_palm(pdf_text)
        st.write("**Summary:**", summary)
        question = st.text_input("Ask a question")
        if question:
            answer = ask_question(pdf_text, question)
            st.write("**Answer:**", answer)

    elif option == "Resume Matcher":
        jd = st.text_area("Paste Job Description")
        if jd:
            result = match_resume_with_jd(pdf_text, jd)
            st.write("**Match Result:**", result)


Overwriting app.py


In [ ]:
!pip install pyngrok


In [ ]:
!ngrok authtoken 30GClqnTjgxenGRtvWa74u7hZCI_33qBGq4durNu16theRUiH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
import threading

def run_streamlit():
    os.system('streamlit run app.py --server.port 8501')

thread = threading.Thread(target=run_streamlit)
thread.start()

In [ ]:
from pyngrok import ngrok
import time
import os

# Terminate any running ngrok tunnels
ngrok.kill()

# Wait a few seconds to make sure Streamlit started
time.sleep(5)

# Create a tunnel to the Streamlit port 8501
public_url = ngrok.connect(8501)
print("Your Streamlit app is live here:",public_url)

Your Streamlit app is live here: NgrokTunnel: "https://f7be598e9191.ngrok-free.app" -> "http://localhost:8501"
